In [105]:
from langchain_groq import ChatGroq

In [106]:
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key=''
)

response = llm.invoke("The first Car was...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [107]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.techtarget.com/WhatIs/feature/Tech-job-boards-to-know")
page_data = loader.load().pop().page_content
print(page_data)













14 tech job boards to know in 2024

























































WhatIs



Search the TechTarget Network






			Browse Definitions
		:

A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
U
V
W
X
Y
Z
#





Login
Register




TechTarget Network
Tech Accelerator
News
2024 IT Salary Survey Results



RSS







WhatIs







Browse Definitions

					HR management










									Business software

View All



									ERP

									Marketing and CX

									Microsoft

									Software applications






										Business software

View All



										ERP

										HR management

										Marketing and CX

										Microsoft

										Software applications






										Cloud computing

View All



										Internet applications

										Internet of Things

										Web services






										Computer science

View All



										Electronics

										Internet technologies

										IT standards and organizations

										Microprocess

In [108]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ###INSTUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the
    following keys: 'role', 'experience', 'skolls' and 'description'.
    Only return in valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
[
  {
    "role": "Software Engineer",
    "experience": "Not specified",
    "skills": "Not specified",
    "description": "Job postings for software engineers can be found on various tech job boards such as Built In, Dice, and HackerEarth."
  },
  {
    "role": "UX/UI Designer",
    "experience": "Not specified",
    "skills": "UX, UI, design",
    "description": "Job postings for UX/UI designers can be found on various tech job boards such as Hired, Ladders, and We Work Remotely."
  },
  {
    "role": "DevOps Engineer",
    "experience": "Not specified",
    "skills": "DevOps, engineering",
    "description": "Job postings for DevOps engineers can be found on various tech job boards such as TechFetch, Wellfound, and Y Combinator."
  },
  {
    "role": "Product Manager",
    "experience": "Not specified",
    "skills": "Product management, sales",
    "description": "Job postings for product managers can be found on various tech job boards such as Levels.fyi, SquadJobs, and W

In [109]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Software Engineer',
  'experience': 'Not specified',
  'skills': 'Not specified',
  'description': 'Job postings for software engineers can be found on various tech job boards such as Built In, Dice, and HackerEarth.'},
 {'role': 'UX/UI Designer',
  'experience': 'Not specified',
  'skills': 'UX, UI, design',
  'description': 'Job postings for UX/UI designers can be found on various tech job boards such as Hired, Ladders, and We Work Remotely.'},
 {'role': 'DevOps Engineer',
  'experience': 'Not specified',
  'skills': 'DevOps, engineering',
  'description': 'Job postings for DevOps engineers can be found on various tech job boards such as TechFetch, Wellfound, and Y Combinator.'},
 {'role': 'Product Manager',
  'experience': 'Not specified',
  'skills': 'Product management, sales',
  'description': 'Job postings for product managers can be found on various tech job boards such as Levels.fyi, SquadJobs, and Whitetruffle.'},
 {'role': 'Data Scientist',
  'experience': 'Not sp

In [110]:
type(json_res)

list

In [111]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [112]:
import chromadb
import uuid

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [113]:
links = collection.query(query_texts=["Experience in python", "Experience in React"], n_results=2).get('metadatas',[])
links
             

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [114]:
job

[{'role': 'Software Engineer',
  'experience': 'Not specified',
  'skills': 'Not specified',
  'description': 'Job postings for software engineers can be found on various tech job boards such as Built In, Dice, and HackerEarth.'},
 {'role': 'UX/UI Designer',
  'experience': 'Not specified',
  'skills': 'UX, UI, design',
  'description': 'Hired is a tech job board that works in reverse, where job posters apply to interview job candidates, and has roles for UX and UI design.'},
 {'role': 'DevOps Engineer',
  'experience': 'Not specified',
  'skills': 'DevOps',
  'description': 'Hired is a tech job board that works in reverse, where job posters apply to interview job candidates, and has roles for DevOps.'},
 {'role': 'Product Manager',
  'experience': 'Not specified',
  'skills': 'Product management',
  'description': 'Hired is a tech job board that works in reverse, where job posters apply to interview job candidates, and has roles for product management.'},
 {'role': 'Sales',
  'experie

In [115]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Chavinda Wijethilake, a business development executive at XYZ Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of XYZ
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase XYZ's portfolio: {link_list}
        Remember you are Chavinda Wijethilake, BDE at XYZ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Solutions for Seamless Business Integration

Dear Hiring Manager,

I came across your job postings for various roles, including Software Engineer, UX/UI Designer, DevOps Engineer, Product Manager, and Sales, among others. As a Business Development Executive at XYZ Software Consulting, I am excited to introduce our company's capabilities in providing tailored solutions to facilitate the seamless integration of business processes through automated tools.

At XYZ, we have a proven track record of empowering numerous enterprises with customized solutions, resulting in scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts is well-versed in developing innovative software solutions that cater to the diverse needs of our clients.

Our portfolio showcases our expertise in various technologies, including machine learning and Python, as demonstrated in our projects featured on https://example.com/ml-python-portfolio and 